In [ ]:
import cv2
import numpy as np
import pickle
import os

In [225]:
feature=cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
cap=cv2.VideoCapture(0)
data=[]
i=0
while True:
    ret, image=cap.read()
    gray=cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    face=feature.detectMultiScale(gray,1.5,5)
    for x,y,w,h in face:
        crop=image[y:y+h,x:x+w,:]
        crop=cv2.resize(crop,(100,100))
        if len(data)<=50:
            data.append(crop)
        i+=1
        cv2.putText(image, str(i), (x+w,y+h), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
        cv2.rectangle(image, (x,y), (x+w,y+h), (255,0,0), 1)
    cv2.imshow("image", image)
    if cv2.waitKey(10)==ord("q") or len(data)==50:
        break

cap.release()
cv2.destroyAllWindows()

name=input("Enter Your Name")
label_name=[name]*50

face_data=np.array(data)
label_name=np.array(label_name)
    

In [ ]:
if not os.path.exists("data_face.pkl"):
    with open("data_face.pkl", "wb") as f:
        pickle.dump(face_data, f)
else:
    with open("data_face.pkl", "rb") as f:
        person_face=pickle.load(f)

    person_face=np.append(person_face,face_data, axis=0)

    with open("data_face.pkl", "wb") as f:
        pickle.dump(person_face, f)

if not os.path.exists("label_name.pkl"):
    with open("label_name.pkl", "wb") as f:
        pickle.dump(label_name, f)
else:
    with open("label_name.pkl", "rb") as f:
        person_name=pickle.load(f)

    person_name=np.append(person_name,label_name, axis=0)

    with open("label_name.pkl", "wb") as f:
        pickle.dump(person_name, f)


In [ ]:
with open("data_face.pkl", "rb") as f:
    person_face=pickle.load(f)

with open("label_name.pkl", "rb") as f:
    person_name=pickle.load(f)

In [228]:
person_face=person_face.reshape(len(person_face),-1)
person_face.shape

(100, 30000)

In [229]:
from sklearn.neighbors import KNeighborsClassifier
model=KNeighborsClassifier(n_neighbors=5)

model.fit(person_face,person_name)

KNeighborsClassifier()

In [242]:
import csv
from datetime import datetime
date_input=input("Enter the Date:")
if not os.path.exists(f"attendence{date_input}.csv"):
    with open(f"attendence{date_input}.csv", "w", newline="") as f:
        writer=csv.writer(f)
        writer.writerow(["NAME","DATE_TIME"])

In [ ]:
cap=cv2.VideoCapture(0)

while True:
    ret, image= cap.read()
    gray=cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    face=feature.detectMultiScale(gray,1.4,5)
    for x,y,w,h in face:
        crop2=image[y:y+h,x:x+w]
        crop2=cv2.resize(crop2,(100,100))
        crop2=np.array(crop2)
        crop2=crop2.reshape(1,-1)
        pre=model.predict(crop2)
        distance, _ = model.kneighbors(crop2)
        print(distance)
        if distance[0][0] > 8500:
            cv2.rectangle(image, (x,y), (x+w,y+h), (0,0,255), 2)
            cv2.putText(image, "Unknown", (x,y-10), cv2.FONT_HERSHEY_SIMPLEX, .75, (255,255,255), 1)
        else:
            cv2.putText(image, str(pre[0]), (x,y-10), cv2.FONT_HERSHEY_SIMPLEX, .75, (255,255,255), 1)
            cv2.rectangle(image, (x,y), (x+w,y+h), (0,255,0), 2)
            
            found=False
            with open(f"attendence{date_input}.csv", "r") as f:
                reader=csv.reader(f)
                for row in reader:
                    if pre[0] in row:
                        found=True
                        break
    
            if not found:
                with open(f"attendence{date_input}.csv", "a", newline="") as f:
                    writer=csv.writer(f)
                    writer.writerow([pre[0],datetime.now().strftime('%Y-%m-%d %H:%M:%S')])

    cv2.imshow("img", image)
    if cv2.waitKey(10)==ord("a"):
        break

cap.release()
cv2.destroyAllWindows()


[[7331.66543154 7385.39680721 7398.06995101 7402.18677149 7444.51260997]]
[[8065.72420803 8139.25168551 8190.17502865 8204.46439446 8216.69751177]]
[[7603.34340932 7701.82575757 7787.1675852  7798.79509924 7813.78691801]]
[[7695.67683833 7776.20961652 7788.20550576 7799.00839851 7809.40618741]]
[[7079.85430641 7154.26488187 7176.29159107 7197.5812604  7250.95269603]]
[[7180.06434512 7252.50529128 7309.67468496 7337.25738952 7341.9809997 ]]
[[7361.93928255 7383.97650321 7393.18530811 7412.85390386 7432.29493494]]
[[6954.6364391  6994.35794051 6996.46889509 7009.94094126 7018.90860462]]
[[7625.39284234 7684.31083442 7717.26505959 7745.89342813 7751.15339804]]
[[7624.69061667 7678.77653015 7682.0476437  7688.89738519 7697.38591991]]
[[7434.63449001 7483.16650891 7502.65146465 7510.93576061 7531.69336869]]
[[7745.40644511 7805.84767978 7844.42929728 7880.85096928 7888.36833572]]
[[7694.22588699 7739.5359034  7744.25348242 7750.56275376 7762.21753109]]
[[7510.78557809 7567.08206642 7587.175